In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station


In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
earliestdate_query_obj = session.query(Measurement.date).order_by(Measurement.date).first()
latestdate_query_obj = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
print(f"Earliest Date in the database: {earliestdate_query_obj[0]} , Latest Date in the database: {latestdate_query_obj[0]}")





In [ ]:
# Calculate the date 1 year ago from the last data point in the database
# Perform a query to retrieve the data and precipitation scores
# Save the query results as a Pandas DataFrame and set the index to the date column
# Sort the dataframe by date

latestdate = dt.datetime.strptime(latestdate_query_obj[0], '%Y-%m-%d')
#latestdate.year
querydate = dt.date(latestdate.year -1, latestdate.month, latestdate.day)
#querydate.year

select_date_prec = [Measurement.date,Measurement.prcp]
queryresult = session.query(*select_date_prec).filter(Measurement.date >= querydate).all()
#queryresult
precipitation = pd.DataFrame(queryresult, columns=['Date','Precipitation'])
precipitation = precipitation.dropna(how='any') 
precipitation = precipitation.sort_values(["Date"], ascending=True)
precipitation = precipitation.set_index("Date")
precipitation.head()


In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data

xaxis = precipitation.index.tolist()
yaxis = precipitation['Precipitation'].tolist()
plt.figure(figsize=(12,10))
plt.bar(xaxis,yaxis,color='b',width = 5,alpha=0.5, align="center",label='precipitation')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
ticks = np.arange(0,300,60)
plt.xticks(ticks)

plt.title(f"Precipitation from {querydate} to {latestdate_query_obj[0]}")
plt.xlabel("date")
plt.ylabel("precipitation")
plt.grid(which='major', axis='both', linestyle='-')
plt.legend()
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precipitation.describe()

In [ ]:
# Design a query to show how many stations are available in this dataset?
number_stations = session.query(Station.id).count()
print (f"Number of Stations = {number_stations}")

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
select_list = [Measurement.station,func.count(Measurement.id)]
active_stat = session.query(*select_list).group_by(Measurement.station).order_by(func.count(Measurement.id).desc()).all()
pd_act_stat = pd.DataFrame(active_stat, columns=['Station','NumberOfRows'])
pd_act_stat

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
Active_station = active_stat[0][0]
select = [func.min(Measurement.tobs),func.max(Measurement.tobs),func.avg(Measurement.tobs)]
mostactivestation = session.query(*select).filter(Measurement.station == Active_station)
pd_mostactivestation = pd.DataFrame(mostactivestation, columns=['MinTemp','MaxTemp','AvgTemp'])
pd_mostactivestation

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

queryresult = session.query(Measurement.tobs).filter(Measurement.station == Active_station).filter(Measurement.date >= querydate).all()
temperatures = list(np.ravel(queryresult))

select = [Station.station,Station.name,Station.latitude,Station.longitude,Station.elevation]
queryresult = session.query(*select).all()
stations_df = pd.DataFrame(queryresult, columns=['Station','Name','Latitude','Longitude','Elevation'])
station = stations_df.loc[stations_df["Station"] == Active_station,"Station"].tolist()[0]
plt.hist(temperatures, bins=12,rwidth=1.0,label='tobs')
plt.grid(axis='both', alpha=0.75)
plt.ylabel('Frequency')
plt.title(f"Temperature from {querydate} to {latestdate_query_obj[0]} observered at {station}")
plt.legend()


![precipitation](Images/station-histogram.png)

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
       
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
start_date = '2017-07-01'
end_date = '2017-07-10'
result = calc_temps(start_date,end_date)[0]
result



In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)

x_pos = [0]
y_pos = [result[1]]
error = [(result[2] - result[0])]
plt.figure(figsize=(4, 7), dpi=80)
plt.bar(x_pos,y_pos,color='coral', yerr=error)
plt.xlim(-0.75,0.75)
plt.title("Trip Avg Temp")
plt.ylabel("Temp (F)")
plt.ylim(0, 100)
plt.tick_params(axis='x',which='both',bottom=False,top=False,labelbottom=False)
plt.grid(which='major', axis='x', linestyle='')
plt.grid(which='major', axis='y', linestyle='-')
plt.show()


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

select = [Station.station,Station.name,Station.latitude,Station.longitude,Station.elevation,func.sum(Measurement.prcp)]
queryresult = session.query(*select).\
    filter(Station.station == Measurement.station).\
    group_by(Measurement.station).\
    filter(Measurement.date >= start_date).\
    filter(Measurement.date <= end_date).\
    order_by(func.sum(Measurement.prcp).desc()).all()
  
stations_df = pd.DataFrame(queryresult, columns=['Station','Name','Latitude','Longitude','Elevation','PrecpAmount'])
stations_df

## Optional Challenge Assignment